Image stack alignment (moving -> fixed)
- For each moving plane, find best (x, y, z, scale) in fixed stack.
- Coarse-to-fine pyramid with template matching + subpixel refinement.
- Outputs per-plane mapping in pixels (x,y) in fixed coords, z index/µm, and scale.

Johannes Larsch 20250916

In [1]:
# Setup & imports
%load_ext autoreload
%autoreload 2
import sys, math, warnings, json
import numpy as np

import os

# Get parent of the notebook dir (project_root)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add sibling directory "src" to sys.path
sys.path.append(os.path.join(project_root, "src"))

import alignSubstackUtils as asu
import saveFunctionalProjections as sfp
from skimage.transform import rotate as sk_rotate

import pandas as pd


from pathlib import Path
import numpy as np
import tifffile as tiff


_HAS_SKIMAGE = True


In [2]:
# === EDIT THIS PATH ===
#START_FOLDER = Path(r"Y:/Danin/imaging/temp/planes/")
START_FOLDER = Path(r"D:/i/danin_tests/temp/planes")



In [8]:

rotation_angle = 140  # degrees, e.g. 0, 90, 180, 270
sfp.save_max_avg_projections(START_FOLDER, rotation_angle)


# Example usage:



Found 6 TIFF stack(s).
[1/6] f38_plane0_mcorrected.tif
[2/6] f38_plane1_mcorrected.tif
[3/6] f38_plane2_mcorrected.tif
[4/6] f38_plane3_mcorrected.tif
[5/6] f38_plane4_mcorrected.tif


<tifffile.TiffFile 'max_projections.tif'> <asarray> failed to reshape (0,) to (), raised ValueError('cannot reshape array of size 0 into shape ()')


[6/6] max_projections.tif


ValueError: zero-size array to reduction operation maximum which has no identity

In [3]:
funcStacks=sfp.collect_tiff_files(START_FOLDER)
funcStacks

[WindowsPath('D:/i/danin_tests/temp/planes/f38_plane0_mcorrected.tif'),
 WindowsPath('D:/i/danin_tests/temp/planes/f38_plane1_mcorrected.tif'),
 WindowsPath('D:/i/danin_tests/temp/planes/f38_plane2_mcorrected.tif'),
 WindowsPath('D:/i/danin_tests/temp/planes/f38_plane3_mcorrected.tif'),
 WindowsPath('D:/i/danin_tests/temp/planes/f38_plane4_mcorrected.tif')]

In [ ]:
# %% [markdown]
# ### Example usage
# Replace the dummy arrays with your actual data (NumPy arrays loaded from TIFF, NIfTI, HDF5, etc.)

# %%


# # Run registration
# df_results = register_moving_stack(
#     fixed_stack,
#     moving_stack,
#     fixed_z_spacing_um=1.0,
#     scale_range=(0.35, 1.0),   # moving is more zoomed-in -> usually < 1.0
#     n_scales=11,
#     z_stride_coarse=4,
#     z_refine_radius=3,
#     pyramid_downscale=2,
#     pyramid_min_size=160,
#     verbose=True,
# )
# display(df_results)

# %% [markdown]
# ### Notes & tips
# - If the moving FOV is substantially smaller, keep `scale_range` < 1 (e.g., 0.2–0.8).
# - If computation is slow, increase `z_stride_coarse` or `pyramid_min_size`.
# - For very noisy data, consider pre-filtering with a small Gaussian before normalization.
# - `ncc_score` near 1.0 indicates a strong match; inspect low scores visually.
# - Coordinates (x_px, y_px) are given in **fixed** image pixels at full resolution.



In [4]:
import tifffile

moving_p = Path(r"Y:/Danin/imaging/temp/planes/projections/avg_projections.tif")
fixed_p = Path(r"Y:/Danin/imaging/temp/f38_anatomy_00001_rotate_8b.tif")
fixed_stack = tifffile.imread(fixed_p)   # shape: (Zf, Yf, Xf), spacing 1 µm
moving_stack = tifffile.imread(moving_p) # shape: (Zm, Ym, Xm), spacing 4–15 µm between planes


df_results = asu.register_moving_stack(
    fixed_stack, 
    moving_stack,
    fixed_z_spacing_um=1.0,     # spacing between planes in the fixed stack (µm)
    scale_range=(0.6, 1.0),   # expected relative zoom (moving→fixed); narrower range boosts SNR
    n_scales=10,                 # number of discrete scales to test between scale_range
    pyramid_downscale=2,      # downscale factor per pyramid level (smaller = finer refinement)
    pyramid_min_size=140,       # stop pyramid when min dimension < this; ensures at least 1 coarse level
    z_stride_coarse=2,          # step size for z-search at coarsest level (higher = faster, lower = more exhaustive)
    z_refine_radius=3,          # number of planes around best z to test at finer levels
    verbose=True,               # print per-plane progress and NCC scores
)

display(df_results)


=== Registering moving plane 0/4 ===
[Coarse] best @ level 0 -> z=124, yx=(35.0,45.0), s=0.711, ncc=0.7004
[Refine L1] z=124, yx=(35.0,45.0), s=0.711, ncc=0.7004

=== Registering moving plane 1/4 ===
[Coarse] best @ level 0 -> z=122, yx=(35.0,42.0), s=0.711, ncc=0.7302
[Refine L1] z=122, yx=(35.0,42.0), s=0.711, ncc=0.7302

=== Registering moving plane 2/4 ===
[Coarse] best @ level 0 -> z=118, yx=(37.0,38.0), s=0.711, ncc=0.7918
[Refine L1] z=118, yx=(37.0,38.0), s=0.711, ncc=0.7918

=== Registering moving plane 3/4 ===
[Coarse] best @ level 0 -> z=112, yx=(39.0,32.0), s=0.711, ncc=0.7998
[Refine L1] z=112, yx=(39.0,32.0), s=0.711, ncc=0.7998

=== Registering moving plane 4/4 ===
[Coarse] best @ level 0 -> z=106, yx=(41.0,27.0), s=0.711, ncc=0.7908
[Refine L1] z=106, yx=(41.0,27.0), s=0.711, ncc=0.7908


,moving_plane,z_index,z_um,y_px,x_px,scale_moving_to_fixed,ncc_score
0,0,124,124.0,68.500000,89.800003,0.711379,0.641108
1,1,122,122.0,70.800003,84.900002,0.711379,0.695683
2,2,118,118.0,74.599998,75.900002,0.711379,0.747606
3,3,112,112.0,77.900002,63.900002,0.711379,0.727211
4,4,106,106.0,80.699997,52.099998,0.711379,0.703861


In [5]:
asu.interactive_checker(fixed_stack, moving_stack, df_results)

Output()

In [9]:
max_p = Path(r"Y:/Danin/imaging/temp/planes/projections/max_projections.tif")

In [15]:
# ===== Usage =====
# fixed_stack: your reference stack (Zf, Yf, Xf) — you likely already have it in memory
# funcStacks: list of paths, one per moving plane (e.g., same order as moving_stack planes)
# Example:
# hyper = build_registered_hyperstack(
#     funcStacks=funcStacks,
#     df_results=df_results,
#     fixed_stack_shape=fixed_stack.shape,
#     dtype=np.float32,
#     save_path="registered_hyperstack_tzyx.tif"   # or None to keep in memory only
# )
# hyper.shape  # -> (100, Zf, Yf, Xf)

save_path = START_FOLDER / "hyperstack_new.tif"
Zf, Yf, Xf = fixed_stack.shape  # (216, 512, 512)


hyper = asu.build_hyperstack_uint8_in_memory(
    funcStacks=funcStacks,
    df_results=df_results,
    fixed_stack_shape=(Zf, Yf, Xf),
    use_percentiles=(1,99.9),                 # or (1, 99) for robust scaling
    save_path=save_path # optional
)

[proc] plane 0: -> z=124, pos=(68.5,89.8), scale=0.711, shape=(100, 364, 364)
[proc] plane 1: -> z=122, pos=(70.8,84.9), scale=0.711, shape=(100, 364, 364)
[proc] plane 2: -> z=118, pos=(74.6,75.9), scale=0.711, shape=(100, 364, 364)
[proc] plane 3: -> z=112, pos=(77.9,63.9), scale=0.711, shape=(100, 364, 364)
[proc] plane 4: -> z=106, pos=(80.7,52.1), scale=0.711, shape=(100, 364, 364)
[scale] global intensity >1 -> 2894.086426 .. 3464.352051
[merge] plane 0 pasted at z=124
[merge] plane 1 pasted at z=122
[merge] plane 2 pasted at z=118
[merge] plane 3 pasted at z=112
[merge] plane 4 pasted at z=106
[saved] D:\i\danin_tests\temp\planes\hyperstack_new.tif


In [16]:
import numpy as np
import tifffile as tiff

def _scale_to_uint8_percentiles(vol, p_low=0.1, p_high=99.1):
    volf = vol.astype(np.float32, copy=False)
    vmin, vmax = np.percentile(volf, [p_low, p_high])
    eps = 1e-6
    out = (volf - vmin) * (255.0 / max(vmax - vmin, eps))
    return np.clip(out, 0, 255).astype(np.uint8, copy=False)

def build_5d_hyperstack_fixed_plus_func(
    fixed_stack,           # (Z, Y, X) — any dtype
    func_hyper_path,       # path to previously saved functional hyperstack (T, Z, Y, X) uint8
    out_path,              # output OME-TIFF path
    T_expected=100
):
    # ----- load / check inputs -----
    Zf, Yf, Xf = fixed_stack.shape
    func = tiff.imread(func_hyper_path)   # expect (T, Z, Y, X), uint8
    if func.ndim != 4:
        raise ValueError(f"Functional hyperstack must be (T,Z,Y,X); got {func.shape}")
    T_func, Z_func, Y_func, X_func = func.shape
    if T_func != T_expected:
        raise ValueError(f"T mismatch: functional has T={T_func}, expected {T_expected}")
    if (Z_func, Y_func, X_func) != (Zf, Yf, Xf):
        raise ValueError(f"Spatial mismatch: func (Z,Y,X)={func.shape[1:]}, fixed={(Zf,Yf,Xf)}")

    # ----- channel 1: fixed stack scaled to uint8 (0.1–99.1 pct), repeated along time -----
    fixed_u8 = _scale_to_uint8_percentiles(fixed_stack, 0.1, 99.1)   # (Z,Y,X) -> uint8
    # Repeat along time to shape (T, Z, Y, X)
    c1 = np.broadcast_to(fixed_u8, (T_expected, Zf, Yf, Xf)).copy()

    # ----- channel 2: functional hyperstack -----
    c2 = func.astype(np.uint8, copy=False)  # should already be uint8 from your previous step

    # ----- stack into (T, C, Z, Y, X) -----
    hyper5 = np.stack([c1, c2], axis=1)  # (T, 2, Z, Y, X)

    # ----- save as OME-TIFF (Fiji-compatible) -----
    # Axes order TCZYX; Bio-Formats/Fiji will read as a 5D hyperstack
    tiff.imwrite(
        out_path,
        hyper5,
        bigtiff=True,
        ome=True,
        metadata={'axes': 'TCZYX'},  # optional: channel names via OME XML is more involved; axes is enough
    )
    print(f"[saved] 5D hyperstack: {out_path}  shape={hyper5.shape}  axes=TCZYX")

# ===== usage =====
# fixed_stack: (Z, Y, X) already in memory
# save_path: path you previously wrote in build_hyperstack_uint8_in_memory (TZYX uint8)
# out_5d: where to save the 5D hyperstack
# Example:
out_5d = START_FOLDER / "fixed_plus_functional_5D.ome.tif"
build_5d_hyperstack_fixed_plus_func(fixed_stack, save_path, out_5d, T_expected=100)

[saved] 5D hyperstack: D:\i\danin_tests\temp\planes\fixed_plus_functional_5D.ome.tif  shape=(100, 2, 216, 512, 512)  axes=TCZYX


In [17]:
import napari
import tifffile as tiff

# Path to the 5D hyperstack you just wrote
#path_5d = START_FOLDER / "fixed_plus_functional_5D.ome.tif"
path_5d = out_5d

# Open with tifffile
stack5d = tiff.imread(path_5d)  # shape = (T, C, Z, Y, X)

print("Loaded shape:", stack5d.shape)  # should be (100, 2, Z, 512, 512)


Loaded shape: (100, 2, 216, 512, 512)


In [20]:
p_mask = START_FOLDER / "f38_anatomy_00001_rotate_mask.tif"

In [23]:
# Split channels: each -> (T, Z, Y, X)
#fixed_TZYX = stack5d[:, 0]
#func_TZYX  = stack5d[:, 1]

mask = tiff.imread(p_mask).astype(bool)  # (Z,Y,X)

# Broadcast mask over T dimension
masked_func = stack5d[:, 1] * mask[None, ...]  # (T,Z,Y,X)
masked_fixed = stack5d[:, 0] * mask[None, ...]


# Set scale: (z, y, x) = (3, 1.1861, 1.1861) um
scale = (3, 1.1861, 1.1861)



viewer = napari.Viewer()

# Add fixed first (background context)
fixed_layer = viewer.add_image(
    masked_fixed,
    name="Fixed",
    colormap="gray",
    blending="translucent",   # 2D/3D-friendly alpha blending
    rendering="translucent",  # for 3D: semi-transparent volume
    opacity=0.28,
    contrast_limits=(0, 255),
    scale=scale,     # attach voxel size
)
# Add functional on top
func_layer = viewer.add_image(
    masked_func,
    name="Functional",
    colormap="green",
    blending="additive",      # bright signal pops through the fixed
    rendering="mip",          # for 3D: max-intensity through the center slab
    opacity=1.0,
    contrast_limits=(0, 255),
    scale=scale,     # attach voxel size
)

viewer.dims.ndisplay = 3      # turn on 3D
# Ensure Functional is above Fixed
if viewer.layers.index(func_layer) < viewer.layers.index(fixed_layer):
    viewer.layers.move(viewer.layers.index(func_layer), len(viewer.layers)-1)

napari.run()

In [27]:
import json
from pathlib import Path
import numpy as np
import napari
import imageio as iio   # pip install imageio imageio-ffmpeg


In [28]:

# ------------------------------
# 1) Save / Load napari settings
# ------------------------------

def save_viewer_preset(viewer: napari.Viewer, folder: Path):
    """
    Save key viewer settings (layer properties, dims, camera) to JSON files in `folder`.
    """
    folder = Path(folder)
    folder.mkdir(parents=True, exist_ok=True)

    # Save per-layer props
    layers_state = []
    for lyr in viewer.layers:
        entry = {
            "name": lyr.name,
            "visible": bool(lyr.visible),
            "opacity": float(lyr.opacity),
            "blending": str(getattr(lyr, "blending", "")),
            "colormap": str(getattr(lyr, "colormap", "")),
            "contrast_limits": list(map(float, getattr(lyr, "contrast_limits", (0, 1)))),
            "rendering": str(getattr(lyr, "rendering", "")),
            "scale": list(map(float, getattr(lyr, "scale", (1, 1, 1)))),
        }
        layers_state.append(entry)

    dims_state = {
        "ndisplay": int(viewer.dims.ndisplay),
        "current_step": list(map(int, viewer.dims.current_step)),  # (… T, Z, Y, X)
        "order": list(map(int, viewer.dims.order)),
    }

    cam = viewer.camera
    camera_state = {
        "zoom": float(cam.zoom),
        "center": list(map(float, getattr(cam, "center", (0, 0, 0)))),
        "angles": list(map(float, getattr(cam, "angles", (0, 0, 0)))),  # (elev, azim, roll)
    }

    with open(folder / "layers.json", "w") as f:
        json.dump(layers_state, f, indent=2)
    with open(folder / "dims.json", "w") as f:
        json.dump(dims_state, f, indent=2)
    with open(folder / "camera.json", "w") as f:
        json.dump(camera_state, f, indent=2)

    print(f"[preset] Saved viewer settings to {folder}")

def load_viewer_preset(viewer: napari.Viewer, folder: Path):
    """
    Restore dims + camera (non-destructive to layer data itself).
    """
    folder = Path(folder)
    if (folder / "dims.json").exists():
        with open(folder / "dims.json") as f:
            dims = json.load(f)
        viewer.dims.ndisplay = dims.get("ndisplay", viewer.dims.ndisplay)
        # optional: restore current_step/order if shapes match
        try:
            viewer.dims.order = dims["order"]
            viewer.dims.current_step = dims["current_step"]
        except Exception:
            pass

    if (folder / "camera.json").exists():
        with open(folder / "camera.json") as f:
            cam = json.load(f)
        viewer.camera.zoom = cam.get("zoom", viewer.camera.zoom)
        if "center" in cam:
            viewer.camera.center = cam["center"]
        if "angles" in cam:
            viewer.camera.angles = cam["angles"]

    print(f"[preset] Loaded viewer settings from {folder}")


In [29]:

# ------------------------------
# 2) Movie recorder from napari
# ------------------------------

def record_movie_from_viewer(
    viewer: napari.Viewer,
    out_path: str = "napari_movie.mp4",
    total_seconds: float = 10.0,
    fps: int = 30,
    step_every_n_frames: int = 2,             # advance time every 2 frames
    size: tuple[int, int] = (640, 640),       # (W,H) — canvas-only render
    rotate_camera: bool = True,
    start_angles: tuple[float,float,float] | None = None,  # (elev, azim, roll)
    end_angles:   tuple[float,float,float] | None = None,
):
    """
    Renders the current viewer scene to a video:
      - Time advances one step every `step_every_n_frames` frames (loops if needed)
      - 30fps default
      - Canvas-only frames
      - Optional gradual camera angle interpolation (3D)
    """
    # Figure out number of frames and time-axis extent
    n_frames = int(round(total_seconds * fps))
    # Heuristic: assume first axis is time (T, Z, Y, X) for your layers
    # If not, set time_axis manually here.
    if len(viewer.dims.current_step) < 4:
        raise RuntimeError("Expected at least 4 dims (T,Z,Y,X).")

    # We look for the time axis by checking which axis has >1 and is first in order by your setup.
    # In your 5D stack added per-channel as separate layers: each layer is (T,Z,Y,X) -> time axis index 0.
    time_axis = 0
    T = viewer.layers[0].data.shape[time_axis]

    # Camera angles
    if rotate_camera and viewer.dims.ndisplay == 3:
        ang0 = tuple(getattr(viewer.camera, "angles", (30.0, 45.0, 0.0))) if start_angles is None else start_angles
        ang1 = (ang0[0]+30, ang0[1]+90, ang0[2]) if end_angles is None else end_angles
    else:
        ang0 = ang1 = tuple(getattr(viewer.camera, "angles", (0.0, 0.0, 0.0)))

    # Prepare writer (H.264/MP4 via imageio-ffmpeg)
    writer = iio.get_writer(out_path, fps=fps, codec="libx264", quality=8)

    try:
        # prime
        orig_angles = tuple(getattr(viewer.camera, "angles", (0.0, 0.0, 0.0)))

        for f in range(n_frames):
            # Advance time every N frames (loop)
            t_idx = (f // step_every_n_frames) % T
            curr = list(viewer.dims.current_step)
            curr[time_axis] = t_idx
            viewer.dims.current_step = tuple(curr)

            # Interpolate camera angles (if 3D)
            if rotate_camera and viewer.dims.ndisplay == 3:
                alpha = f / max(n_frames - 1, 1)
                viewer.camera.angles = [
                    (1 - alpha) * ang0[i] + alpha * ang1[i] for i in range(3)
                ]

            # Grab canvas-only screenshot at requested size (W,H)
            frame = viewer.screenshot(canvas_only=True, flash=False, size=size)
            writer.append_data(frame)

        # restore original camera
        viewer.camera.angles = orig_angles

    finally:
        writer.close()

    print(f"[movie] Saved {out_path}  ({n_frames} frames @ {fps} fps ≈ {n_frames/fps:.2f}s)")

# ------------------------------
# Usage
# ------------------------------

# 1) Save current settings so you can reuse from now on
napariPresetsFolder =  START_FOLDER / "napari_presets"
#START_FOLDER = Path("napari_presets")
save_viewer_preset(viewer, napariPresetsFolder)

# (Later, you can restore with:)
# load_viewer_preset(viewer, napariPresetsFolder)

# 2) Record a movie (defaults: 10s, 30fps, time advance every 2 frames, 640x640)
record_movie_from_viewer(
    viewer,
    out_path = START_FOLDER / "hyperstack_demo.mp4",
    total_seconds=10.0,         # change this if you want longer/shorter
    fps=30,
    step_every_n_frames=2,      # time++ every 2 frames (=> 15 time-steps/sec)
    size=(640, 640),            # near 512x512, better for players/codecs
    rotate_camera=True,         # set False for a static camera
    # start_angles=(30, 45, 0), # optional: set explicit start/end if you want
    # end_angles=(60, 135, 0),
)


[preset] Saved viewer settings to D:\i\danin_tests\temp\planes\napari_presets
[movie] Saved D:\i\danin_tests\temp\planes\hyperstack_demo.mp4  (300 frames @ 30 fps ≈ 10.00s)
